In [22]:
######################################################
#
# Utility functions
#
######################################################
import numpy as np
from sklearn.metrics import mean_squared_error
import copy
from numpy.linalg import qr  as qr

class tsUtils:
    
    def updateSVD(D, uk, sk, vk):
        vk = vk.T
        m = vk.shape[1]
        d = m+D.shape[1]
        D_k = np.dot(np.dot(D.T, uk), np.diag(1 / sk))
        vkh = np.zeros([len(sk), d])
        vkh[:, :m] = vk
        vkh[:, m:d] = D_k.T

        return uk, sk, vkh.T


    def updateSVD2(D, uk, sk, vk):
        vk = vk.T
        k,m = vk.shape
        n,p = D.shape
        # memory intensive? nxn dot nxp
        D_h = np.dot(np.eye(n)- np.dot(uk,uk.T),D)
        # Qr of n X p matrix ~ relatively easy
        Qd,Rd = qr(D_h)

        A_h = np.zeros([p+k,p+k])
        A_h[:k,:k] = np.diag(sk)
        A_h[:k,k:k+p] = np.dot(uk.T,D)
        A_h[k:k+p, k:k+p] = Rd
        # SVD of p+k X p+k matrix ~ relatively easy
        ui, si, vi = np.linalg.svd(A_h, full_matrices=False)
        uk_h = ui[:,:k]
        sk_h = si[:k]
        vk_h = vi[:k,:]

        sk_u = sk_h

        # matirx mult. n X (k+p) by (k+p) X k
        #uk_u = np.dot(np.concatenate((uk,Qd),1),uk_h)
        uk_u = np.zeros([n, k+p])
        uk_u[:,:k] = uk
        uk_u[:, k:k+p] = Qd
        uk_u = np.dot(uk_u,uk_h)

        vk_u = np.zeros([m+p,k+p])
        vk_u[:m,:k] = vk.T
        vk_u[m:m+p, k:k+p] = np.eye(p)

        vk_2 = np.dot(vk_u,vk_h.T)
        return uk_u, sk_u, vk_2

    def arrayToMatrix(npArray, nRows, nCols):

        if (type(npArray) != np.ndarray):
            raise Exception('npArray is required to be of type np.ndarray')

        if (nRows * nCols != len(npArray)):
            raise Exception('(nRows * nCols) must equal the length of npArray')

        return np.reshape(npArray, (nCols, nRows)).T


    def matrixFromSVD(sk, Uk, Vk, probability=1.0):
        return (1.0/probability) * np.dot(Uk, np.dot(np.diag(sk), Vk.T))

    def pInverseMatrixFromSVD(sk, Uk, Vk, probability=1.0):
        s = copy.deepcopy(sk)
        for i in range(0, len(s)):
            if (s[i] > 0.0):
                s[i] = 1.0/s[i]

        p = 1.0/probability
        return tsUtils.matrixFromSVD(s, Vk, Uk, probability=p)


    def rmse(array1, array2):
        return np.sqrt(mean_squared_error(array1, array2))


    def rmseMissingData(array1, array2):

        if (len(array1) != len(array2)):
            raise Exception('lengths of array1 and array2 must be the same.')

        subset1 = []
        subset2 = []
        for i in range(0, len(array1)):
            if np.isnan(array1[i]):
                subset1.append(array1[i])
                subset2.append(array2[i])

        return tsUtils.rmse(subset1, subset2)


    def normalize(array, max, min):

        diff = 0.5*(min + max)
        div = 0.5 * (max - min)

        array = (array - diff)/div
        return array

    def unnormalize(array, max, min):

        diff = 0.5*(min + max)
        div = 0.5*(max - min)

        array = (array *div) + diff
        return array


    def randomlyHideValues(array, pObservation):

        count = 0
        for i in range(0, len(array)):
            if (np.random.uniform(0, 1) > pObservation):
                array[i] = np.nan
                count +=1 

        p_obs = float(count)/float(len(array))
        return (array, 1.0 - p_obs)

# chooses rows of the matrix according to pObservationRow
# hide stretches of data with the longestStretch being the max entries hidden in a row
# gap should ideally be the number of columns of the matrix this array will be converted in to
    def randomlyHideConsecutiveEntries(array, pObservationRow, longestStretch, gap):

        n = len(array)
        valuesToHide = int((1.0 - pObservationRow) * n)

        count = 0
        countStart = 0
        i = 0
        while (i < n):
            # decide if this point is the start of a randomly missing run
            if (np.random.uniform(0, 1) > pObservationRow):
                countStart +=1

                # now decide how many consecutive values go missing and where to start
                toHide = longestStretch #int(np.random.uniform(0, 1) * longestStretch)
                startingIndex = i + int(np.random.uniform(0, 1) * (gap - toHide))

                if (toHide + startingIndex >  (i + gap)):
                    toHide = (i + gap) - startingIndex

                array[startingIndex: startingIndex + toHide] = np.nan * np.zeros(toHide)
                
                count += toHide

                valuesToHide -= toHide

                if (valuesToHide <= 0):
                    break

            # ensure there is some space between consecutive runs
            i += gap

        p_obs = float(count)/float(n)

        return (array, 1.0 - p_obs)


    # following is taken from: https://stackoverflow.com/questions/6518811/interpolate-nan-values-in-a-numpy-array
    def nanInterpolateHelper(array):
        """Helper to handle indices and logical indices of NaNs.

        Input:
            - y, 1d numpy array with possible NaNs
        Output:
            - nans, logical indices of NaNs
            - index, a function, with signature indices= index(logical_indices),
            to convert logical indices of NaNs to 'equivalent' indices
        Example:
            >>> # linear interpolation of NaNs
            >>> nans, x= nan_helper(y)
            >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
        """
        (nans, x) = (np.isnan(array), lambda z: z.nonzero()[0])
        array[nans] = np.interp(x(nans), x(~nans), array[~nans])
        return array



In [23]:
#SVD Wrapper
class SVD:

    def __init__(self, matrix, method='numpy'):
        if (type(matrix) != np.ndarray):
            raise Exception('SVDWrapper required matrix to be of type np.ndarray')

        self.methods = ['numpy']

        self.matrix = matrix
        self.U = None
        self.V = None
        self.s = None
        (self.N, self.M) = np.shape(matrix)

        if (method not in self.methods):
            print("The methods specified (%s) if not a valid option. Defaulting to numpy.linalg.svd" %method)
            self.method = 'numpy'

        else:
            self.method = method

    # perform the SVD decomposition
    # method will set the self.U and self.V singular vector matrices and the singular value array: self.s
    # U, s, V can then be access separately as attributed of the SVDWrapper class
    def decompose(self):
        # default is numpy's linear algebra library
        (self.U, self.s, self.V) = np.linalg.svd(self.matrix, full_matrices=False)

        # correct the dimensions of V
        self.V = self.V.T

    # get the top K singular values and corresponding singular vector matrices
    def decomposeTopK(self, k):

        # if k is 0 or less, just return empty arrays
        if (k < 1):
            return ([], [], [])

        # if k > the max possible singular values, set it to be that value
        elif (k > np.min([self.M, self.N])):
            k = np.min([self.M, self.N])

        if ((self.U is None) | (self.V is None) | (self.s is None)):
            self.decompose() # first perform the full decomposition

        sk = self.s[0:k]
        Uk = self.U[:, 0:k]
        Vk = self.V[:, 0:k]

        return (sk, Uk, Vk)

    # get the matrix reconstruction using top K singular values
    # if returnMatrix = True, then return the actual matrix, else return sk, Uk, Vk
    def reconstructMatrix(self, kSingularValues, returnMatrix=False):

        (sk, Uk, Vk) = self.decomposeTopK(kSingularValues)
        if (returnMatrix == True):
            return tsUtils.matrixFromSVD(sk, Uk, Vk)
        else:
            return (sk, Uk, Vk)






In [38]:
#ALS Wrapper
######################################################
#
# Alternating Least Squares
#
######################################################
import time
class ALS:

    def __init__(self, matrix, method='als'):
        if (type(matrix) != np.ndarray):
            raise Exception('ALSWrapper required matrix to be of type np.ndarray')

        self.methods = ['als']

        self.matrix = matrix

        (self.N, self.M) = np.shape(matrix)

        self.W = np.zeros([self.N, self.M])
        mask = np.isnan(self.matrix)
        self.W[mask == True] = 0.0
        self.W[mask == False] = 1.0
        self.W = self.W.astype(np.float64, copy=False)

        self.matrix[mask == True] = 0.0

        if (method not in self.methods):
            print("The methods specified (%s) if not a valid option. Defaulting to ALS" %method)
            self.method = 'als'

        else:
            self.method = method

    # run the ALS algorithm
    # k is the number of factors
    def decompose(self, k, lambda_, iterations, tol):

        middleVal = 0.5 * (np.max(self.matrix) + np.min(self.matrix))

        # initialize randomly
        U = middleVal * np.random.rand(self.N, k) 
        V = middleVal * np.random.rand(k, self.M)

        # fix max iterations
        maxIter = iterations

        pastError = np.inf
        for ii in range(maxIter):
            # first U matrix with V fixed
            for u, Wu in enumerate(self.W):
                left = np.linalg.pinv(np.dot(V, np.dot(np.diag(Wu), V.T)) + lambda_ * np.eye(k))
                right = np.dot(V, np.dot(np.diag(Wu), self.matrix[u].T))
                U[u] = np.dot(left, right).T

                    #np.linalg.solve(np.dot(V, np.dot(np.diag(Wu), V.T)) + lambda_ * np.eye(k),
                              # np.dot(V, np.dot(np.diag(Wu), self.matrix[u].T))).T

            # now V matrix with U fixed
            for i, Wi in enumerate(self.W.T):
                left = np.linalg.pinv(np.dot(U.T, np.dot(np.diag(Wi), U)) + lambda_ * np.eye(k))
                right = np.dot(U.T, np.dot(np.diag(Wi), self.matrix[:, i]))
                V[:,i] = np.dot(left, right).T

                #np.linalg.solve(np.dot(U.T, np.dot(np.diag(Wi), U)) + lambda_ * np.eye(k),
                          #       np.dot(U.T, np.dot(np.diag(Wi), self.matrix[:, i])))
            
            # compute MSE
            err = self.getError(self.matrix, U, V, self.W)

            # break if difference is less than tol
            deltaErr = np.abs(err - pastError)
            if (deltaErr < tol):
                break
            else:
                pastError = err

            if (ii%10 == 0):
                print("Iteration %d, Err = %0.4f, DeltaErr = %0.4f" %(ii+1, pastError, deltaErr))

        print('Total Iterations = %d' %(ii+1))
        return (U,V)
        


    # get the matrix reconstruction using k factors and missing data
    def reconstructMatrix(self, k, lambda_, returnMatrix=True, iterations=1000, tol=1e-6):

        (Uk, Vk) = self.decompose(k, lambda_, iterations, tol)
        if (returnMatrix == True):
            return np.dot(Uk, Vk)
        else:
            return (Uk, Vk)


    # MSE function for the ALS algorithm
    def getError(self, Q, U, V, W):
        return np.mean((W * (Q - np.dot(U, V)))**2)


In [16]:
######################################################
#
# The Time Series Model based on SVD
#
######################################################
import copy
import numpy as np
import pandas as pd

class SVDModel(object):

    # seriesToPredictKey:       (string) the time series of interest (key)
    # kSingularValuesToKeep:    (int) the number of singular values to retain
    # N:                        (int) the number of rows of the matrix for each series
    # M:                        (int) the number of columns for the matrix for each series
    # probObservation:          (float) the independent probability of observation of each entry in the matrix
    # svdMethod:                (string) the SVD method to use (optional)
    # otherSeriesKeysArray:     (array) an array of keys for other series which will be used to predict 
    # includePastDataOnly:      (Boolean) defaults to True. If this is set to False, 
    #                               the time series in 'otherSeriesKeysArray' will include the latest data point.
    #                               Note: the time series of interest (seriesToPredictKey) will never include 
    #                               the latest data-points for prediction
    def __init__(self, seriesToPredictKey, kSingularValuesToKeep, N, M, probObservation=1.0, svdMethod='numpy', otherSeriesKeysArray=[], includePastDataOnly=True, start = 0, TimesUpdated = 0, TimesReconstructed =0 ):

        self.seriesToPredictKey = seriesToPredictKey
        self.otherSeriesKeysArray = otherSeriesKeysArray
        self.includePastDataOnly = includePastDataOnly

        self.N = N
        self.M = M
        self.start = start
        self.TimesUpdated = TimesUpdated
        self.TimesReconstructed = TimesReconstructed
        self.kSingularValues = kSingularValuesToKeep
        self.svdMethod = svdMethod

        self.Uk = None
        self.Vk = None
        self.sk = None
        self.matrix = None
        self.lastRowObservations = None
        self.Ukw = None
        self.Vkw = None
        self.skw = None
        self.p = probObservation

        self.weights = None

    # run a least-squares regression of the last row of self.matrix and all other rows of self.matrix
    # sets and returns the weights
    # DO NOT call directly
    def _computeWeights(self):       

        ### This is now the same as ALS
        ## this is an expensive step because we are computing the SVD all over again 
        ## however, currently, there is no way around it since this is NOT the same matrix as the full
        ## self.matrix, i.e. we have fewer (or just one less) rows

        if (self.lastRowObservations is None):
            raise Exception('Do not call _computeWeights() directly. It should only be accessed via class methods.')

        # need to decide how to produce weights based on whether the N'th data points are to be included for the other time series or not
        # for the seriesToPredictKey we only look at the past. For others, we could be looking at the current data point in time as well.
        
        matrixDim1 = (self.N * len(self.otherSeriesKeysArray)) + self.N-1
        matrixDim2 = np.shape(self.matrix)[1]
        eachTSRows = self.N

        if (self.includePastDataOnly == False):
            newMatrix = self.matrix[0:matrixDim1, :]

        else:
            matrixDim1 = ((self.N - 1) * len(self.otherSeriesKeysArray)) + self.N-1
            eachTSRows = self.N - 1

            newMatrix = np.zeros([matrixDim1, matrixDim2])

            rowIndex = 0
            matrixInd = 0

            while (rowIndex < matrixDim1):
                newMatrix[rowIndex: rowIndex + eachTSRows] = self.matrix[matrixInd: matrixInd +eachTSRows]

                rowIndex += eachTSRows
                matrixInd += self.N

        svdMod = SVD(newMatrix, method='numpy')
        (self.skw, self.Ukw, self.Vkw) = svdMod.reconstructMatrix(self.kSingularValues, returnMatrix=False)

        newMatrixPInv = tsUtils.pInverseMatrixFromSVD(self.skw, self.Ukw, self.Vkw, probability=self.p)
        self.weights = np.dot(newMatrixPInv.T, self.lastRowObservations.T)

    # return the imputed matrix
    def denoisedDF(self):
        setAllKeys = set(self.otherSeriesKeysArray)
        setAllKeys.add(self.seriesToPredictKey)

        single_ts_rows = self.N
        dataDict = {}
        rowIndex = 0
        for key in self.otherSeriesKeysArray:

            dataDict.update({key: self.matrix[rowIndex*single_ts_rows: (rowIndex+1)*single_ts_rows, :].flatten('F')})
            rowIndex += 1

        dataDict.update({self.seriesToPredictKey: self.matrix[rowIndex*single_ts_rows: (rowIndex+1)*single_ts_rows, :].flatten('F')})

        return pd.DataFrame(data=dataDict)

    def denoisedTS(self, ind, range = True):

        NewColsDenoised = tsUtils.matrixFromSVD(self.sk, self.Uk, self.Vk, probability=self.p).flatten(1)
        if range:
            assert len(ind) == 2
            return NewColsDenoised[ind[0]:ind[1]]
        else:

            return NewColsDenoised[ind]


    def denoisedDFNew(self,D,updateMethod = 'folding-in', missingValueFill = True):
        assert (len(D) % self.N == 0)
        p = len(D)/self.N
        self.updateSVD(D,updateMethod)
        NewColsDenoised = tsUtils.matrixFromSVD(self.sk, self.Uk, self.Vk[-p:,:], probability=self.p)

        return NewColsDenoised.flatten(1)


    # this internal method assigns the data (provided to fit()) to the class variables to help with computations
    # if missingValueFill = True, then we will impute with the middle value
    def _assignData(self, keyToSeriesDF, missingValueFill=True):

        setAllKeys = set(self.otherSeriesKeysArray)
        setAllKeys.add(self.seriesToPredictKey)

        if (len(set(keyToSeriesDF.columns.values).intersection(setAllKeys)) != len(setAllKeys)):
            raise Exception('keyToSeriesDF does not contain ALL keys provided in the constructor.')

        if (missingValueFill == True):
            # impute with the least informative value (middle)
            max = np.nanmax(keyToSeriesDF)

            min = np.nanmin(keyToSeriesDF)
            diff = 0.5*(min + max)
            keyToSeriesDF = keyToSeriesDF.fillna(value=diff)

        T = self.N * self.M
        for key in setAllKeys:
            if (len(keyToSeriesDF[key]) < T):
                raise Exception('All series (columns) provided must have length >= %d' %T)


        # initialize the matrix of interest
        single_ts_rows = self.N
        matrix_cols = self.M
        matrix_rows = (len(setAllKeys) * single_ts_rows)

        self.matrix = np.zeros([matrix_rows, matrix_cols])

        seriesIndex = 0
        for key in self.otherSeriesKeysArray: # it is important to use the order of keys set in the model
            self.matrix[seriesIndex*single_ts_rows: (seriesIndex+1)*single_ts_rows, :] = tsUtils.arrayToMatrix(keyToSeriesDF[key][-1*T:].values, single_ts_rows, matrix_cols)
            seriesIndex += 1

        # finally add the series of interest at the bottom
       # tempMatrix = tsUtils.arrayToMatrix(keyToSeriesDF[self.seriesToPredictKey][-1*T:].values, self.N, matrix_cols)
        self.matrix[seriesIndex*single_ts_rows: (seriesIndex+1)*single_ts_rows, :] = tsUtils.arrayToMatrix(keyToSeriesDF[self.seriesToPredictKey][-1*T:].values, single_ts_rows, matrix_cols)
        
        # set the last row of observations
        self.lastRowObservations = copy.deepcopy(self.matrix[-1, :])


    # keyToSeriesDictionary: (Pandas dataframe) a key-value Series (time series)
    # Note that the keys provided in the constructor MUST all be present
    # The values must be all numpy arrays of floats.
    # This function sets the "de-noised" and imputed data matrix which can be accessed by the .matrix property
    def fit(self, keyToSeriesDF):

        # assign data to class variables

        self._assignData(keyToSeriesDF, missingValueFill=True)
        # now produce a thresholdedthresholded/de-noised matrix. this will over-write the original data matrix
        svdMod = SVD(self.matrix, method='numpy')
        (self.sk, self.Uk, self.Vk) = svdMod.reconstructMatrix(self.kSingularValues, returnMatrix=False)
        self.matrix = tsUtils.matrixFromSVD(self.sk, self.Uk, self.Vk, probability=self.p)
        # set weights
        self._computeWeights()



    def updateSVD(self,D, method = 'folding-in', missingValueFill = True):
        assert (len(D) % self.N == 0)
        if (missingValueFill == True):
            # impute with the least informative value (middle)
            max = np.nanmax(D)
            if np.isnan(max): max = 0
            min = np.nanmin(D)
            if np.isnan(min): min = 0
            diff = 0.5*(min + max)
            D[np.isnan(D)] = diff

        D = D.reshape([self.N,int(len(D)/self.N)])

        assert D.shape[0] == self.N
        assert D.shape[1] <= D.shape[0]
        if method == 'UP':
            self.Uk, self.sk, self.Vk = tsUtils.updateSVD2(D, self.Uk, self.sk, self.Vk)
            self.M = self.Vk.shape[0]
            self.Ukw, self.skw, self.Vkw = tsUtils.updateSVD2(D[:-1,:], self.Ukw, self.skw, self.Vkw)
        elif method == 'folding-in':
            self.Uk, self.sk, self.Vk = tsUtils.updateSVD(D, self.Uk, self.sk ,self.Vk )
            self.M = self.Vk.shape[0]
            self.Ukw, self.skw, self.Vkw = tsUtils.updateSVD(D[:-1, :], self.Ukw, self.skw, self.Vkw)
        # elif method == 'Full':
        #     raise ValueError
        #     self.matrix = np.concatenate((self.matrix,D),1)
        #     U, S, V = np.linalg.svd(self.matrix, full_matrices=False)
        #     self.sk = S[0:self.kSingularValues]
        #     self.Uk = U[:, 0:self.kSingularValues]
        #     self.Vk = V[0:self.kSingularValues,:]
        #     self.Vk = self.Vk.T
        #     self.M = self.Vk.shape[0]
        else:
            raise ValueError
        self.TimesUpdated +=1

        newMatrixPInv = tsUtils.pInverseMatrixFromSVD(self.skw, self.Ukw, self.Vkw, probability=self.p)
        self.lastRowObservations = np.append(self.lastRowObservations,D[-1,:])
        self.weights = np.dot(newMatrixPInv.T, self.lastRowObservations.T)



    # otherKeysToSeriesDFNew:     (Pandas dataframe) needs to contain all keys provided in the model;
    #                           If includePastDataOnly was set to True (default) in the model, then:
    #                               each series/array MUST be of length >= self.N - 1
    #                               If longer than self.N - 1, then the most recent self.N - 1 points will be used
    #                           If includePastDataOnly was set to False in the model, then:
    #                               all series/array except seriesToPredictKey MUST be of length >= self.N (i.e. includes the current), 
    #                               If longer than self.N, then the most recent self.N points will be used
    #
    # predictKeyToSeriesDFNew:   (Pandas dataframe) needs to contain the seriesToPredictKey and self.N - 1 points past points.
    #                           If more points are provided, the most recent self.N - 1 points are selected.   
    #
    # bypassChecks:         (Boolean) if this is set to True, then it is the callee's responsibility to provide
    #                           all required series of appropriate lengths (see above).
    #                           It is advised to leave this set to False (default).         
    def predict(self, otherKeysToSeriesDFNew, predictKeyToSeriesDFNew, bypassChecks=False):

        nbrPointsNeeded = self.N - 1
        if (self.includePastDataOnly == False):
            nbrPointsNeeded = self.N

        if (bypassChecks == False):

            if (self.weights is None):
                raise Exception('Before predict() you need to call "fit()" on the model.')

            if (len(set(otherKeysToSeriesDFNew.columns.values).intersection(set(self.otherSeriesKeysArray))) < len(set(self.otherSeriesKeysArray))):
                raise Exception('keyToSeriesDFNew does not contain ALL keys provided in the constructor.')

            for key in self.otherSeriesKeysArray:
                points = len(otherKeysToSeriesDFNew[key])
                if (points < nbrPointsNeeded):
                    raise Exception('Series (%s) must have length >= %d' %(key, nbrPointsNeeded))

            points = len(predictKeyToSeriesDFNew[self.seriesToPredictKey])
            if (points < self.N - 1):
                raise Exception('Series (%s) must have length >= %d' %(self.seriesToPredictKey, self.N - 1))

        newDataArray = np.zeros((len(self.otherSeriesKeysArray) * nbrPointsNeeded) + self.N - 1)
        indexArray = 0
        for key in self.otherSeriesKeysArray:
            newDataArray[indexArray: indexArray + nbrPointsNeeded] = otherKeysToSeriesDFNew[key][-1*nbrPointsNeeded: ].values

            indexArray += nbrPointsNeeded

        # at last fill in the time series of interest
        newDataArray[indexArray:] = predictKeyToSeriesDFNew[self.seriesToPredictKey][-1*(self.N - 1):].values

        # dot product
        return np.dot(self.weights, newDataArray)







In [39]:
######################################################
#
# The Time Series Model based on ALS
#
######################################################

class ALSModel(SVDModel):

    # seriesToPredictKey:       (string) the time series of interest (key)
    # kFactors:    				(int) number of factors (similar to the kSingularValues of the parent class)
    # N:                        (int) the number of rows of the matrix for each series
    # M:                        (int) the number of columns for the matrix for each series
    # probObservation:          (float) the independent probability of observation of each entry in the matrix
    # otherSeriesKeysArray:     (array) an array of keys for other series which will be used to predict 
    # includePastDataOnly:      (Boolean) defaults to True. If this is set to False, 
    #                               the time series in 'otherSeriesKeysArray' will include the latest data point.
    #                               Note: the time series of interest (seriesToPredictKey) will never include 
    #                               the latest data-points for prediction
    def __init__(self, seriesToPredictKey, kFactors, N, M, probObservation=1.0, otherSeriesKeysArray=[], includePastDataOnly=True):

        super(ALSModel, self).__init__(seriesToPredictKey, kFactors, N, M, probObservation=probObservation, svdMethod='numpy', otherSeriesKeysArray=otherSeriesKeysArray, includePastDataOnly=includePastDataOnly)

    # run a least-squares regression of the last row of self.matrix and all other rows of self.matrix
    # sets and returns the weights
    # DO NOT call directly
    def _computeWeights(self):   
        if (self.lastRowObservations is None):
            raise Exception('Do not call _computeWeights() directly. It should only be accessed via class methods.')

        # need to decide how to produce weights based on whether the N'th data points are to be included for the other time series or not
        # for the seriesToPredictKey we only look at the past. For others, we could be looking at the current data point in time as well.
        
        matrixDim1 = (self.N * len(self.otherSeriesKeysArray)) + self.N-1
        matrixDim2 = np.shape(self.matrix)[1]
        eachTSRows = self.N

        if (self.includePastDataOnly == False):
        	newMatrix = self.matrix[0:matrixDim1, :]
    
        else:
            matrixDim1 = ((self.N - 1) * len(self.otherSeriesKeysArray)) + self.N-1
            eachTSRows = self.N - 1

            newMatrix = np.zeros([matrixDim1, matrixDim2])

            rowIndex = 0
            matrixInd = 0
            print(eachTSRows)
            while (rowIndex < matrixDim1):
            	newMatrix[rowIndex: rowIndex + eachTSRows] = self.matrix[matrixInd: matrixInd +eachTSRows]
            	rowIndex += eachTSRows
            	matrixInd += self.N
        self.weights = np.dot(np.linalg.pinv(newMatrix).T, self.lastRowObservations.T)


	# keyToSeriesDictionary: (Pandas dataframe) a key-value Series (time series)
    # Same as the parent class (SVDModel)
    def fit(self, keyToSeriesDF):

        # assign data to class variables
        super(ALSModel, self)._assignData(keyToSeriesDF, missingValueFill=False)

        self.max = np.nanmax(self.matrix)
        self.min = np.nanmin(self.matrix)

        # now use ALS to produce an estimated matrix
        alsMod = ALS(self.matrix, method='als')
        (U, V) = alsMod.reconstructMatrix(self.kSingularValues, 0.0, returnMatrix=False, tol=1e-9)

        self.matrix = np.dot(U, V)

        self.matrix[self.matrix > self.max] = self.max
        self.matrix[self.matrix < self.min] = self.min

        # we need to assign some values to the lastRowObservations where there are still NaNs
        # impute those with the ALS-estimated/iputed values
        for i in range(0, len(self.lastRowObservations)):
        	if (np.isnan(self.lastRowObservations[i])):
        		self.lastRowObservations[i] = self.matrix[-1, i]

        # set weights (same as the parent class now that we have the SVD of the ALS-estimated matrix)
        self._computeWeights()
	
	# return the imputed matrix, same as the parent class (SVDModel)
    def denoisedDF(self):

    	return super(ALSModel, self).denoisedDF()

	# same params as the predict() method of the parent class (SVDModel)       
    def predict(self, otherKeysToSeriesDFNew, predictKeyToSeriesDFNew, bypassChecks=False):

    	return super(ALSModel, self).predict(otherKeysToSeriesDFNew, predictKeyToSeriesDFNew, bypassChecks)

    def updateSVD(self, D):
        return super(ALSModel, self).updateSVD(self, D)